# Hyperparameter Optimization mit hyperas

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import h5py
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc

from hyperopt import Trials, STATUS_OK, tpe

from keras.models import Sequential
from keras.layers import Dense, Dropout, LeakyReLU, Activation
from keras.regularizers import l2, l1
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.models import load_model
from sklearn.metrics import confusion_matrix, classification_report
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from keras.callbacks import EarlyStopping
from keras.models import load_model

from hyperas import optim
from hyperas.distributions import choice, uniform, normal

Using TensorFlow backend.


### Data preparation

In [3]:
def data_Bert():
    news = pd.read_csv('../data/mixed_news/news_dataset.csv')
    news = news.dropna(subset=['title','content'])
    news = news[news.content != ' ']
    news = news[news.title != ' ']
    with h5py.File('encoded_data/title_encode.h5', 'r') as hf:
        title_encode = hf['title_encode'][:]
    title_NN = pd.DataFrame(data = title_encode)
    x_train, x_test, y_train, y_test = train_test_split(title_NN,news.label, test_size=0.33, stratify=news.label,
                                                        random_state=42)
    LE = LabelEncoder()
    LE.fit(["fake","real"])
    y_train = LE.transform(y_train)
    y_test = LE.transform(y_test)
    return x_train, y_train, x_test, y_test

In [4]:
def data_bow():
    x_train = np.genfromtxt("../build/preprocessed/bow_X_train.txt",unpack=True)
    y_train = np.genfromtxt("../build/preprocessed/bow_y_train.txt", unpack=True)
    x_test  = np.genfromtxt("../build/preprocessed/bow_X_test.txt",unpack=True)
    y_test  = np.genfromtxt("../build/preprocessed/bow_y_test.txt", unpack=True)
    return x_train, y_train, x_test, y_test

In [ ]:
x_train = np.genfromtxt("../build/preprocessed/bow_X_train.txt",unpack=True)
y_train = np.genfromtxt("../build/preprocessed/bow_y_train.txt", unpack=True)
x_test  = np.genfromtxt("../build/preprocessed/bow_X_test.txt",unpack=True)
y_test  = np.genfromtxt("../build/preprocessed/bow_y_test.txt", unpack=True)
x_train

## Model creation

In [5]:
def model_(x_train, y_train, x_test, y_test):
    dim = x_train.shape[1]
    model = Sequential()
    model.add(Dense(int({{normal(1000, 200)}}), input_dim=dim))
    model.add(Activation('relu'))
    
    if {{choice(['three', 'four'])}} == 'four':
        model.add(Dense(int({{normal( 1000, 200)}})))
        model.add(Activation('relu'))
        
    model.add(Dense({{randint(300)}}))
    model.add(Activation('relu'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(loss='binary_crossentropy', metrics=['accuracy'],
                  optimizer='adam')

    early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=2)

    result = model.fit(x_train, y_train,
              batch_size=64,
              epochs=30,
              verbose=2,
              validation_split=0.3,
              callbacks=[early_stop])
    validation_acc = np.amax(result.history['val_acc']) 
    print('Best validation acc of epoch:', validation_acc)
    return {'loss': -validation_acc, 'status': STATUS_OK, 'model': model}

In [6]:
def model_training(x_train, y_train, x_test, y_test):
    dim = x_train.shape[1]
    model = Sequential()
    model.add(Dense(1000, input_dim=dim))
    model.add(Activation('relu'))
    model.add(Dense(1000))
    model.add(Activation('relu'))    
    model.add(Dense(10))
    model.add(Activation('relu'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(loss={{choice(['hinge','binary_crossentropy','squared_hinge'])}}, metrics=['accuracy'],
                  optimizer={{choice(['adam','AdaDelta','Adagrad'])}})

    early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=2)

    result = model.fit(x_train, y_train,
              batch_size=64,
              epochs=30,
              verbose=2,
              validation_split=0.3,
              callbacks=[early_stop])
    validation_acc = np.amax(result.history['val_acc']) 
    print('Best validation acc of epoch:', validation_acc)
    return {'loss': -validation_acc, 'status': STATUS_OK, 'model': model}

In [1]:
def model_regularization(x_train, y_train, x_test, y_test):
    dim = x_train.shape[1]
    model = Sequential()
    model.add(Dense(1000, kernel_regularizer=l1({{uniform(0,0.1)}}), input_dim=dim))
    model.add(Activation('relu'))
    model.add(Dense(1000),kernel_regularizer=l2({{uniform(0,0.1)}}))
    model.add(Activation('relu'))    
    model.add(Dense(10),kernel_regularizer=l2({{uniform(0,0.1)}}))
    model.add(Activation('relu'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(loss='binary_crossentropy', metrics=['accuracy'],
                  optimizer='adam')

    early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=2)

    result = model.fit(x_train, y_train,
              batch_size=64,
              epochs=30,
              verbose=2,
              validation_split=0.3,
              callbacks=[early_stop])
    validation_acc = np.amax(result.history['val_acc']) 
    print('Best validation acc of epoch:', validation_acc)
    return {'loss': -validation_acc, 'status': STATUS_OK, 'model': model}

## Optimization with hyperopt
Algorithm: Tree of Parzen Estimators

In [ ]:
trials = Trials()
best_run, best_model = optim.minimize(model=model_structure,
                                      data=data_bow,
                                      algo=tpe.suggest,
                                      max_evals=50,
                                      trials=trials,
                                     notebook_name='Hyperopt')
print("Best performing model chosen hyper-parameters:")
print(best_run)
best_model.save('../model/best_Hyperopt_NN_Bert.hdf5')

>>> Imports:
#coding=utf-8

try:
    import numpy as np
except:
    pass

try:
    import matplotlib.pyplot as plt
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import h5py
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.metrics import roc_curve, auc
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from keras.layers import Dense, Dropout, LeakyReLU, Activation
except:
    pass

try:
    from keras.callbacks import TensorBoard, ModelCheckpoint
except:
    pass

try:
    from keras.models import load_model
except:
    pass

try:
    from sklearn.metrics import confusion_matrix, classification_report
except:
    pass

try:
    from keras.utils import np_utils
except:
    pass

try:
    from sklearn.preprocessing import LabelEncoder
except:
    pass

try:
    from keras.callbacks i

## Evaluation of best model

In [ ]:
X_train, Y_train, X_test, Y_test = data()
best_model = load_model('../model/best_Hyperopt_NN_Bert.hdf5')
y_pred = best_model.predict(X_test, batch_size=64, verbose=1)
y_pred_bool = np.round(y_pred)
print(classification_report(Y_test, y_pred_bool))
print(confusion_matrix(Y_test, y_pred_bool,
                       labels=[0,1]))
plt.imshow(confusion_matrix(Y_test, y_pred_bool,
                            labels=[0,1]))
plt.tight_layout()
plt.colorbar()
plt.xticks(range(2), ["fake", "real"])
plt.yticks(range(2), ["fake", "real"])
plt.show()
#plt.savefig("../build/plots/Bert/cnfsn_mtx_Hyperopt_bert_title_nn.pdf")
plt.close()

plt.hist(y_pred[Y_test == 0],label="fake", alpha = 0.4, color = "r")
plt.hist(y_pred[Y_test == 1],label = "real",alpha = 0.4, color = "b")
plt.xlabel("Prediction Probability")
plt.legend()
plt.show()
#plt.savefig("../build/plots/Bert/prob_Hyperopt_bert_title_nn.pdf")
plt.close()

fpr = dict()
tpr = dict()
roc_auc = dict()
fpr, tpr, _ = roc_curve(Y_test, y_pred)
roc_auc = auc(fpr, tpr)

plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()
#plt.savefig("../build/plots/Bert/roc_Hyperopt_bert_title_nn.pdf")
plt.close()

In [ ]:
trials.trials

In [ ]:
best_model.get_config()